# B2B Ecommerce Fraud dataset

## B2B courier charges accuracy analysis focuses on assessing the accuracy of fees charged by courier companies for the delivery of goods in B2B transactions. The aim is to ensure that companies are billed appropriately for the services provided by courier companies.

Let's start this task by importing the necessary Python libraries and the dataset.

In [ ]:
import pandas as pd
import numpy as np

order_report = pd.read_csv("/content/Order Report.csv")
sku_master = pd.read_csv("/content/SKU Master.csv")
pincode_mapping = pd.read_csv("/content/pincodes.csv")
courier_invoice =  pd.read_csv("/content/Invoice.csv")
courier_company_rates = pd.read_csv("/content/Courier Company - Rates.csv")


print("Order Report")
print(order_report.head())
print("\nSKU Master:")
print(sku_master.head())
print("\nPincode Mapping:")
print(pincode_mapping.head())
print("\nCourier Invoice:")
print(courier_invoice.head())
print("\nCourier Company rates:")
print(courier_company_rates.head())




Order Report
   ExternOrderNo            SKU  Order Qty  Unnamed: 3  Unnamed: 4
0     2001827036  8904223818706        1.0         NaN         NaN
1     2001827036  8904223819093        1.0         NaN         NaN
2     2001827036  8904223819109        1.0         NaN         NaN
3     2001827036  8904223818430        1.0         NaN         NaN
4     2001827036  8904223819277        1.0         NaN         NaN

SKU Master:
             SKU  Weight (g)  Unnamed: 2  Unnamed: 3  Unnamed: 4
0  8904223815682         210         NaN         NaN         NaN
1  8904223815859         165         NaN         NaN         NaN
2  8904223815866         113         NaN         NaN         NaN
3  8904223815873          65         NaN         NaN         NaN
4  8904223816214         120         NaN         NaN         NaN

Pincode Mapping:
   Warehouse Pincode  Customer Pincode Zone  Unnamed: 3  Unnamed: 4
0             121003            507101    d         NaN         NaN
1             121003        

Now let's have a look if any of the data contains missing values:


In [ ]:
# Check for missing values
print("\nMissing values in Website Order Report:")
print(order_report.isnull().sum())
print("\nMissing values in SKU Master:")
print(sku_master.isnull().sum())
print("\nMissing values in Pincode Mapping:")
print(pincode_mapping.isnull().sum())
print("\nMissing values in Courier Invoice:")
print(courier_invoice.isnull().sum())
print("\nMissing values in courier company rates:")
print(courier_company_rates.isnull().sum())


Missing values in Website Order Report:
ExternOrderNo      0
SKU                0
Order Qty          0
Unnamed: 3       400
Unnamed: 4       400
dtype: int64

Missing values in SKU Master:
SKU            0
Weight (g)     0
Unnamed: 2    66
Unnamed: 3    66
Unnamed: 4    66
dtype: int64

Missing values in Pincode Mapping:
Warehouse Pincode      0
Customer Pincode       0
Zone                   0
Unnamed: 3           124
Unnamed: 4           124
dtype: int64

Missing values in Courier Invoice:
AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64

Missing values in courier company rates:
fwd_a_fixed         0
fwd_a_additional    0
fwd_b_fixed         0
fwd_b_additional    0
fwd_c_fixed         0
fwd_c_additional    0
fwd_d_fixed         0
fwd_d_additional    0
fwd_e_fixed         0
fwd_e_additional    0
rto_a_fixed        

Now let's clean the data

In [ ]:
#  Remove unnamed columns from the Website Order Report DataFrame
order_report = order_report.drop(columns=["Unnamed: 3", "Unnamed: 4"])

# Remove unnamed columns from the SKU Master DataFrame
sku_master = sku_master.drop(columns=["Unnamed: 2","Unnamed: 3", "Unnamed: 4"])

# Remove unnamed columns from the pincode Mapping Dataframe
pincode_mapping = pincode_mapping.drop(columns=["Unnamed: 3", "Unnamed: 4"])

Now let's merge the order report and SKU master  datasets according to the common SKU column:

In [ ]:
# Merge the Order Report and SKU Master based on SKU
merged_data = pd.merge(order_report, sku_master, on='SKU')
print(merged_data.head())

   ExternOrderNo            SKU  Order Qty  Weight (g)
0     2001827036  8904223818706        1.0         127
1     2001827036  8904223819093        1.0         150
2     2001827036  8904223819109        1.0         100
3     2001827036  8904223818430        1.0         165
4     2001827036  8904223819277        1.0         350


In [ ]:
# The "ExternOrderNo" is Nothing but "Order Id" in other dataset. Let's rename this column:
# Rename the "ExternOrderNo" column to "Order Id"
merged_data = merged_data.rename(columns={'ExternOrderNo': 'Order ID'})

Now let's merge the courier invoice and pincode mapping dataset:


In [ ]:
abc_courier = pincode_mapping.drop_duplicates(subset=['Customer Pincode'])
courier_abc = courier_invoice[['Order ID', 'Customer Pincode', 'Type of Shipment']]
pincodes = courier_abc.merge(abc_courier, on='Customer Pincode')
print(pincodes.head())


     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


Now let's merge the pincodes with the main dataframe:

In [ ]:
merged2 = merged_data.merge(pincodes, on='Order ID')

In [ ]:
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000

In [ ]:
merged2 = merged2.drop(columns=['Weight (Kgs)'])

In [ ]:
courier_invoice.tail(20)

,AWB Code,Order ID,Charged Weight,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Weight Slab Charged by Courier Company
104,1091117437293,2001808739,1.63,121003,342012,d,Forward charges,179.8,2.0
105,1091117437864,2001808832,2.47,121003,334001,d,Forward charges,224.6,2.5
106,1091117437890,2001808837,0.67,121003,302031,d,Forward charges,90.2,1.0
107,1091117438074,2001808883,0.72,121003,302012,d,Forward charges,90.2,1.0
108,1091117611501,2001808992,0.72,121003,342014,d,Forward charges,90.2,1.0
109,1091117613962,2001809270,0.68,121003,324005,d,Forward charges,90.2,1.0
110,1091117803511,2001809934,0.82,121003,302001,d,Forward charges,90.2,1.0
111,1091117804314,2001810125,0.66,121003,302004,d,Forward charges,90.2,1.0
112,1091117805390,2001810281,0.68,121003,302018,d,Forward charges,90.2,1.0
113,1091117806263,2001810549,1.86,121003,302017,d,Forward charges,179.8,2.0


Now let's calculate the weight slabs:


In [ ]:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

merged2['Weight Slab (KG)'] = merged2['Weights (Kgs)'].apply(weight_slab)
courier_invoice['Weight Slab Charged by Courier Company'] = (courier_invoice['Charged Weight']).apply(weight_slab)

Now let's rename the columns to prepare the desired dataframe.

In [ ]:
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})
merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per ABC'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per ABC'})

Now let's calculate the expected charges:

In [ ]:
total_expected_charge = []

for _, row in merged2.iterrows():
    fwd_category = 'fwd_' + row['Delivery Zone As Per ABC']
    fwd_fixed = courier_company_rates.at[0, fwd_category + '_fixed']
    fwd_additional = courier_company_rates.at[0, fwd_category + '_additional']
    rto_category = 'rto_' + row['Delivery Zone As Per ABC']
    rto_fixed = courier_company_rates.at[0, rto_category + '_fixed']
    rto_additional = courier_company_rates.at[0, rto_category + '_additional']

    weight_slab = row['Weight Slab As Per ABC']

    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * fwd_additional)
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (weight_slab - 0.5) / 0.5)
        total_expected_charge.append(fwd_fixed + additional_weight * (fwd_additional + rto_additional))
    else:
        total_expected_charge.append(0)

merged2['Expected Charge as per ABC'] = total_expected_charge
print(merged2.head(20))



      Order ID            SKU  Order Qty  Weight (g)  Customer Pincode  \
0   2001827036  8904223818706        1.0         127            173213   
1   2001827036  8904223819093        1.0         150            173213   
2   2001827036  8904223819109        1.0         100            173213   
3   2001827036  8904223818430        1.0         165            173213   
4   2001827036  8904223819277        1.0         350            173213   
5   2001827036  GIFTBOX202002        1.0         500            173213   
6   2001827036  GIFTBOX202002        1.0         500            173213   
7   2001827036  8904223818638        2.0         137            173213   
8   2001827036     SACHETS001        1.0          10            173213   
9   2001825261  8904223819024        4.0         112            517128   
10  2001825261  8904223819291        4.0         112            517128   
11  2001825261  8904223818638        3.0         137            517128   
12  2001825261  8904223818669        1

Now let's merge it with the courier invoice to display the final dataframe:

In [ ]:
merged_output = merged2.merge(courier_invoice, on='Order ID')
print(merged_output.head())

     Order ID            SKU  Order Qty  Weight (g)  Customer Pincode_x  \
0  2001827036  8904223818706        1.0         127              173213   
1  2001827036  8904223819093        1.0         150              173213   
2  2001827036  8904223819109        1.0         100              173213   
3  2001827036  8904223818430        1.0         165              173213   
4  2001827036  8904223819277        1.0         350              173213   

  Type of Shipment_x  Warehouse Pincode_x Delivery Zone As Per ABC  \
0    Forward charges               121003                        e   
1    Forward charges               121003                        e   
2    Forward charges               121003                        e   
3    Forward charges               121003                        e   
4    Forward charges               121003                        e   

   Weights (Kgs)  Weight Slab As Per ABC  Expected Charge as per ABC  \
0          0.127                     0.5                

Now let's calculate th differences in charges and expected charges for each order:

In [ ]:
df_diff = merged_output
df_diff['Difference (Rs.)'] = df_diff['Billing Amount (Rs.)'] - df_diff['Expected Charge as per ABC']

df_new = df_diff[['Order ID', 'Difference (Rs.)', 'Expected Charge as per ABC']]

print(df_new.head())

     Order ID  Difference (Rs.)  Expected Charge as per ABC
0  2001827036              61.3                        56.6
1  2001827036              61.3                        56.6
2  2001827036              61.3                        56.6
3  2001827036              61.3                        56.6
4  2001827036              61.3                        56.6


Now let’s summarize the accuracy of B2B courier charges based on the charged prices and expected prices:

In [ ]:
# Calculate the total orders in each category
total_correctly_charged = len(df_new[df_new['Difference (Rs.)'] == 0])
total_overcharged = len(df_new[df_new['Difference (Rs.)'] > 0])
total_undercharged = len(df_new[df_new['Difference (Rs.)'] < 0])

# Calculate the total amount in each category
amount_overcharged = abs(df_new[df_new['Difference (Rs.)'] > 0]['Difference (Rs.)'].sum())
amount_undercharged = df_new[df_new['Difference (Rs.)'] < 0]['Difference (Rs.)'].sum()
amount_correctly_charged = df_new[df_new['Difference (Rs.)'] == 0]['Expected Charge as per ABC'].sum()

# Create a new DataFrame for the summary
summary_data = {'Description': ['Total Orders where ABC has been correctly charged',
                                'Total Orders where ABC has been overcharged',
                                'Total Orders where ABC has been undercharged'],
                'Count': [total_correctly_charged, total_overcharged, total_undercharged],
                'Amount (Rs.)': [amount_correctly_charged, amount_overcharged, amount_undercharged]}

df_summary = pd.DataFrame(summary_data)

print(df_summary)

                                         Description  Count  Amount (Rs.)
0  Total Orders where ABC has been correctly charged     12         507.6
1        Total Orders where ABC has been overcharged    382       33750.5
2       Total Orders where ABC has been undercharged      7        -165.2


We can also visualize the proportion of errors as shown below:

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Pie(labels=df_summary['Description'],
                            values=df_summary['Count'],
                            textinfo='label+percent',
                            hole=0.4))
fig.update_layout(title='Proportion')

fig.show()